# Exploratory Data Analysis for Vancouver AirBnB Dataset

The purpose of this exploratory data analysis (EDA) is to understand:
- **Preliminary feature selection**: Our dataset contains many features, many of which are categorical variables. Which variables should we select (based on our understanding of AirBnB and collective personal experience) should we select to begin training our model with? 
- **The composition of our dataset**: What are the types of hosts and properties that are represented? This will help us understand the validity of the model we will develop to predict an appropriate nightly price for a new AirBnB property in Vancouver. 
- **Missing data and data preprocessing required**: How much missing data is there? Is it an acceptable amount and how will we treat missing data? What is the range of values in our features? Will we have to preprocess to normalize these values for our model to work well?


## Imports and reading in the data

Note that the data has not been provided in the GitHub repository. The `data.py` script has to be run in order to download the data to your local machine to reproduce this notebook. 

In [1]:
# Imports
import numpy as np
import pandas as pd
import altair as alt
import os

import pandas_profiling
from sklearn.model_selection import train_test_split

In [5]:
#Loading the data set
data = pd.read_csv('data/listings.csv.gz')

In [6]:
data.head()

,id,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,...,instant_bookable,is_business_travel_ready,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,10080,https://www.airbnb.com/rooms/10080,20191109094845,2019-11-09,D1 - Million Dollar View 2 BR,"Stunning two bedroom, two bathroom apartment. ...","Bed setup: 2 x queen, option to add up to 2 tw...","Stunning two bedroom, two bathroom apartment. ...",none,NaN,...,t,f,strict_14_with_grace_period,f,f,43,43,0,0,0.16
1,13188,https://www.airbnb.com/rooms/13188,20191109094845,2019-11-09,Garden level studio in ideal loc.,Garden level studio suite with garden patio - ...,Very Close (3min walk) to Nat Bailey baseball ...,Garden level studio suite with garden patio - ...,none,The uber hip Main street area is a short walk ...,...,t,f,moderate,f,f,1,1,0,0,1.90
2,13357,https://www.airbnb.com/rooms/13357,20191109094845,2019-11-09,! Wow! 2bed 2bath 1bed den Harbour View Apartm...,Very spacious and comfortable with very well k...,"Mountains and harbour view 2 bedroom,2 bath,1 ...",Very spacious and comfortable with very well k...,none,Amanzing bibrant professional neighbourhood. C...,...,f,f,strict_14_with_grace_period,t,t,3,1,2,0,0.48
3,13490,https://www.airbnb.com/rooms/13490,20191109094845,2019-11-09,Vancouver's best kept secret,This apartment rents for one month blocks of t...,"Vancouver city central, 700 sq.ft., main floor...",This apartment rents for one month blocks of t...,none,"In the heart of Vancouver, this apartment has ...",...,f,f,strict_14_with_grace_period,f,f,1,1,0,0,0.82
4,14267,https://www.airbnb.com/rooms/14267,20191109094845,2019-11-09,EcoLoft Vancouver,"The Ecoloft is located in the lovely, family r...",West Coast Modern Laneway House Loft: We call ...,"The Ecoloft is located in the lovely, family r...",none,We live in the centre of the city of Vancouver...,...,t,f,strict_14_with_grace_period,f,f,1,1,0,0,0.28


## Preliminary feature selection

Let's understand the number of data points and features our dataset has.

In [7]:
print("Our dataset has", data.shape[0],"number of rows and", data.shape[1],"number of columns")

Our dataset has 6181 number of rows and 106 number of columns


As we expected, our dataset has many more features than we would want to start with in our baseline model. Simpler models are better and more interpretable so we want to prune our dataset to only include features that:
1. We would reasonably expect to know when considering setting up a new AirBnB property
2. We believe matters most to nightly price based on our understanding of the AirBnB booking system

We decided on three categories of features: 
1. **Host-related information** such as host response rate to requests, whether the host is a superhost, whether the host identity has been verified
2. **Property-related information** such as property type, the neighborhood, number of people who can be accommodated, number of bathrooms, bedrooms and beds
3. **Booking-related information** such as whether the property can be instantly booked, the cancellation policy



There are also several potential target variables that relate to pricing we can select including monthly, weekly and nightly price. Our chosen target variable is nightly price since we believe that is the one that is most commonly used when booking a property.

We select these features and our target variable from our data below. Then we split our data into X and y training and test sets before doing further EDA. 

In [8]:
# Preprosessing part

#List of chosen features
selected = ['id', 'host_id', 'host_response_rate', 'host_is_superhost', 'property_type', 
             'host_identity_verified', 'neighbourhood_cleansed', 'instant_bookable', 'cancellation_policy', 
            'accommodates', 'bathrooms', 'bedrooms', 'beds', 'price'
           ]

data.drop(data.columns.difference(selected), axis = 1, inplace = True)

#Changing format of price column 
data.price = data.price.str.replace('$', '').str.replace(',', '').astype(float)

#Changing format of host response rate column
data['host_response_rate'] = data['host_response_rate'].str.rstrip('%').astype('float') / 100.0

#Splitting data into test and train 
X_train, X_test, y_train, y_test = train_test_split(data.drop(columns = 'price'), 
                                                    data[['price']], 
                                                    test_size = 0.2,
                                                   random_state = 123)

X_train['price'] = y_train

In [9]:
X_train.head()

,id,host_id,host_response_rate,host_is_superhost,host_identity_verified,neighbourhood_cleansed,property_type,accommodates,bathrooms,bedrooms,beds,instant_bookable,cancellation_policy,price
3210,28462644,19201130,0.8,f,f,Downtown Eastside,Condominium,4,2.0,2.0,2.0,f,moderate,199.0
1475,16599114,10617862,1.0,f,t,Grandview-Woodland,House,2,2.5,1.0,1.0,f,moderate,81.0
5753,38856942,41546711,1.0,f,t,West Point Grey,House,1,1.0,1.0,0.0,t,strict_14_with_grace_period,36.0
3822,32503415,64045071,1.0,f,t,Downtown,Condominium,5,1.0,2.0,2.0,f,strict_14_with_grace_period,40.0
4869,36278131,103182440,1.0,f,f,Downtown,Apartment,4,1.0,1.0,1.0,t,strict_14_with_grace_period,399.0


## Composition of our dataset

### Understanding our features

To begin our EDA, we want to use Pandas Profiling Report. This report does not render on GitHub.com. A copy of the report can be found in the analysis folder as 'profile.html'. We want to understand the composition of the type of hosts, properties and bookings in our dataset. 

In [10]:
profile = pandas_profiling.ProfileReport(X_train)
profile

In [11]:
profile.to_file(output_file="profile.html")

Here are some select insights: 

**Hosts:**
- About 1/3 of hosts' identities are verified and 1/3 of hosts' are superhosts. This is a pretty good spread and the composition of the hosts in our dataset is not extremely unbalanced. 
- Vast majority of hosts have high response rates. As we fit models to our dataset, this may be a less important metric since there is not much differentiation etween hosts. This parameter has a high number of missing values (12.5% of the dataset). We can either choose to not include this variable or understand what missing data means (e.g., does it mean that it's a new host who has never responded to a request) and treat the data appropriately.
- A handful of hosts have missing identity and superhost. Since there aren't too many, we can probably remove data points with missing host data from our dataset.
- There are around ~3500 hosts for ~5000 properties. This means that some hosts have multiple properties. This does not impact our analysis since we're interested in how to price a future AirBnB property. 

**Properties:** 
- The histograms of how many people each property can accommodate, the number of bathrooms, bedrooms and beds suggest that there is a variety of properties in our dataset. For example, the most common number of people that properties can accommodate is 2 but there is a healthy range from 1 to 8.
- Property type is also sufficiently varied with the majority being houses, apartments and condos. There are 15 additional categories to this. It may be worth it to combine certain categories so that we reduce the number of features as a part of preprocessing. 
- There are 23 neighborhoods with the majority of properties located downtown. While intuitively, neighborhood is a probably a good predictor for price, this may be too many categories. We should consider merging nearby neighborhoods to reduce the number of categories.

**Bookings:**
- The most common cancellation policy is "strict_14_with_grace_period" and there is sufficient distribution in "moderate" and "flexible" for our dataset to learn from. There are only 2 datapoints with a "strict" cancellation policy - our model may not predict well with this policy since there are so few datapoints to learn from. 
- There is almost an equal split between whether a property is instantly bookable or not.


In general, there are no glaring issues with using our dataset for machine learning. There is some preprocessing to be done with missing data and feature selection / engineering to simplify the model. 

### Understanding our target variable

We also want to understand the distribution of prices (our target variable) in our training set. We plot a histogram below.

In [12]:
alt.Chart(X_train).mark_bar(clip = True).encode(
    alt.X('price:Q',
         scale=alt.Scale(domain=(0, 1000)),
         bin=alt.Bin(extent=[0, 1000], step=25),
         title='Nightly price'),
    alt.Y('count()', title='No. of properties')
).properties(width=600, height = 300, title = 'Number of properties by nightly price (between $0 and $1000)')

alt.Chart(...)

Majority of properties are priced between $50 to $200 per night. There is a long right tail to this distribution indicating that there are few properties at high nightly prices. As we create a model that suggests / predicts a price of a new AirBnB property, we have to be conscious of the fact that the training set has had more data to learn from at prices towards the centre of the distribution. 

## Further analysis on neighborhoods and property type

We want to understand how nightly price varies with neighborhoods and property type. To do this, we have labeled our nightly pricing as low (<100 CAD), mid (100-300 CAD), high (300-500 CAD) and exceptional (above 500 CAD). We create a dataframe below to assist with our analysis. 

In [13]:
price_data_labels = X_train[['price', 'neighbourhood_cleansed', 'property_type']]
price_data_labels['label'] = pd.cut(price_data['price'], bins=[0, 100, 300, 500, 13000], 
                               include_lowest=True, labels=['low', 'mid', 'high', 'exceptional'])
price_data_labels

NameError: name 'price_data' is not defined

### Understanding price by neighborhood

To understand the relationship between nightly price and neighborhoods, we analyze the distribution of the number of properties by neighborhood and price point. 

In [14]:
alt.Chart(price_data_labels).mark_rect().encode(
    alt.X('neighbourhood_cleansed:N', title="Neighborhoods"),
    alt.Y('price:Q', bin=alt.Bin(extent=[0, 1000], step=50), title="Nightly price ($ CAD)"),
    alt.Color('count()')
)

alt.Chart(...)

**Select insights:**
- Some neighborhoods do not have any properties listed above a certain price point. For instance, Strathcona and Killarney have no properties listed above 350/night. Most neighborhoods do not have any properties listed above 600/night. This lack of training examples for properties of certain prices in certain neighborhoods has implications on our model's ability to predict properties in these "edge cases". 
- Downtown and Kitsilano have some of the highest priced properties, with listings almost consistently up to 1000/night.
- This analysis further emphasizes the need to combine neighborhoods to improve the performance of our model.

Below is a tool that shows the distribution of price categories by neighborhood. You can toggle by neighborhood (does not render correctly on GitHub.com). 

In [15]:
# Choose the neighborhood in the drop down to see the distribution of nightly prices. 

select_box = alt.binding_select(options=list(price_data_labels['neighbourhood_cleansed'].unique()))
selection = alt.selection_single(name='Select', fields=['neighbourhood_cleansed'], bind=select_box)

b_line = alt.Chart(price_data_labels).mark_bar().encode(
    alt.X('label:N', title = 'Price category',
          sort=["low", "mid", "high", "exceptional"]),
    alt.Y('count()', title = 'No. of AirBnB listings'),
    color = 'neighbourhood_cleansed').properties(width=600, height = 200)

b_line.add_selection(
    selection
).transform_filter(
    selection
).configure_legend(
    orient='bottom'
).properties(
    title = "Total number of AirBnB's across neighbourhoods in Vancouver per price category"
)


alt.Chart(...)

Below is a tool that displays the number of properties in each neighborhood by a selected price category. This is meant to be interactive. 

In [16]:
# Table for total count of Airbnb listings per price category in a neighbourhood. Can be used for exploratory purposes. 
by_neighboorhood = price_data_labels.groupby(['label', 'neighbourhood_cleansed']).count().reset_index()
by_neighboorhood = by_neighboorhood.drop(['property_type'], axis = 1).rename(columns={"price": "count"})

# Selection
by_neighboorhood.loc[by_neighboorhood['label'] == 'low']

KeyError: 'label'

### Understanding price by property type

To understand the relationship between nightly price and property type, we analyze the distribution of the number of properties by property type and price point. 

In [17]:
alt.Chart(price_data_labels).mark_rect().encode(
    alt.X('property_type:N', title="Property Type"),
    alt.Y('price:Q', bin=alt.Bin(extent=[0, 1000], step=50), title="Nightly price ($ CAD)"),
    alt.Color('count()')
)

alt.Chart(...)

**Select insights:**
- Other than houses, condos and apartments, other categories have very sparse data, especially across price points. In particular, Aparthotel, Bed and breakfast, Boat, Boutique hotel, Cabin, Cottage, Hotel, Timeshare and Tinyhouse are problematic. These categories should probably be collapsed into "Other" as a part of preprocessing. 
- The model would be able to best predict on unseen house, apartment and condo properties since there is the most data to learn from across price points.

Below is a tool that shows the distribution of price categories by property type. You can toggle by property (does not render correctly on GitHub.com). 

In [34]:
# Choose the property type in the drop down to see the distribution of nightly prices. 

select_box_2 = alt.binding_select(options=list(price_data_labels['property_type'].unique()))
selection_2 = alt.selection_single(name='Select', fields=['property_type'], bind=select_box_2)

b_line_2 = alt.Chart(price_data_labels).mark_bar().encode(
    alt.X('label:N', title = 'Price category'),
    alt.Y('count()', title = 'No. of AirBnB listings'),
    color = 'property_type').properties(width=600, height = 200)

b_line_2.add_selection(
    selection_2
).transform_filter(
    selection_2
).configure_legend(
    orient='bottom'
).properties(
    title = "Total number of AirBnB's for a property type in Vancouver per price category"
)

alt.Chart(...)

In [261]:
data = pd.read_csv('data/listings.csv.gz')

In [262]:
# Preprosessing part

#List of chosen features
selected = ['id', 'host_id', 'host_response_rate', 'host_is_superhost', 'property_type', 
             'host_identity_verified', 'neighbourhood_cleansed', 'instant_bookable', 'cancellation_policy', 
            'accommodates', 'bathrooms', 'bedrooms', 'beds', 'price'
           ]

data.drop(data.columns.difference(selected), axis = 1, inplace = True)

#Changing format of price column 
data.price = data.price.str.replace('$', '').str.replace(',', '').astype(float)

#Changing format of host response rate column
data['host_response_rate'] = data['host_response_rate'].str.rstrip('%').astype('float') / 100.0

# drop nas

data = data.dropna()

#Splitting data into test and train 
X_train, X_test, y_train, y_test = train_test_split(data.drop(columns = 'price'), 
                                                    data[['price']], 
                                                    test_size = 0.2,
                                                   random_state = 123)

X_train['price'] = y_train

In [263]:
from sklearn.preprocessing import LabelEncoder

# process and drop columns that can't be encoded
processor = LabelEncoder()
X_train = X_train.drop(columns = "host_is_superhost")
X_train[['property_type']] = processor.fit_transform(X_train[['property_type']])
X_train = X_train.drop(columns = "host_identity_verified")
X_train[['neighbourhood_cleansed']] = processor.fit_transform(X_train[['neighbourhood_cleansed']])
X_train = X_train.drop(columns = "instant_bookable")
X_train[['cancellation_policy']] = processor.fit_transform(X_train[['cancellation_policy']])

X_train = X_train.drop(columns = ["id", "host_id", "price"])

# remove column headers
X_train.columns = [''] * len(X_train.columns)
y_train.columns = [''] * len(y_train.columns)

# change format to array
X_train = np.asarray(X_train)
y_train = np.asarray(y_train)

In [264]:
# preprocessing for testing data

# process and drop columns that can't be encoded
processor = LabelEncoder()
X_test = X_test.drop(columns = "host_is_superhost")
X_test[['property_type']] = processor.fit_transform(X_test[['property_type']])
X_test = X_test.drop(columns = "host_identity_verified")
X_test[['neighbourhood_cleansed']] = processor.fit_transform(X_test[['neighbourhood_cleansed']])
X_test = X_test.drop(columns = "instant_bookable")
X_test[['cancellation_policy']] = processor.fit_transform(X_test[['cancellation_policy']])

X_test = X_test.drop(columns = ["id", "host_id"])

# remove column headers
X_test.columns = [''] * len(X_test.columns)
y_test.columns = [''] * len(y_test.columns)

# change format to array
X_test = np.asarray(X_test)
y_test = np.asarray(y_test)

In [265]:
def fit_and_report(model, X, y, Xv, yv, mode = 'regression'):
    """
    fits and trains a model on the dataset and reports the train and validation error to use as a baseline for the case without pruning features,
    if the mode is regression then it returns the training and validation mean squared error,
    if the mode is classification then it returns the training and validation error
    """
    model.fit(X, y)
    if mode.lower().startswith('regress'):
        errors = [mean_squared_error(y, model.predict(X)), mean_squared_error(yv, model.predict(Xv))]
    if mode.lower().startswith('classif'):
        errors = [1 - model.score(X,y), 1 - model.score(Xv,yv)]        
    return errors

In [ ]:
# number of features to select RFE 
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.feature_selection import RFE
from sklearn.svm import SVC, SVR

train_MSE = []
test_MSE = []

for i in range(1,20):
    lr = LinearRegression()
    rfe = RFE(estimator = lr, n_features_to_select = i)
    rfe.fit(X_train, y_train)
    
    X_train_sel = X_train[:, rfe.support_]
    X_test_sel = X_test[:, rfe.support_]

    clf = SVR(kernel = 'rbf')
    errors = fit_and_report(clf, X_train_sel, y_train, X_test_sel, y_test, mode = "classification")
    
    train_MSE.append(errors[0])
    test_MSE.append(errors[1])

In [ ]:
import matplotlib.pyplot as plt

num_of_feat = np.arange(1, 20, 1)
train_MSE

plt.plot(num_of_feat, train_MSE)
plt.xlabel('number of features')
plt.ylabel('training MSE')
plt.show()

In [ ]:
import matplotlib.pyplot as plt

num_of_feat = np.arange(1, 20, 1)
test_MSE

plt.plot(num_of_feat, test_MSE)
plt.xlabel('number of features')
plt.ylabel('validation MSE')
plt.show()

In [ ]:
class ForwardSelection:
    def __init__(self, model, 
                 min_features=None, 
                 max_features=None, 
                 scoring=None, 
                 cv=None):
        """
        initializes the ForwardSelection model
        
        parameters:
        - max_features: sets the maximum number of features to be selected
        - min_features: sets the minimum number of features to be selected
        - model: the model used to generate the cross validation score to obtain the error for a given set of features
        - cv: the number of times we split the data, fit a model, and compute the score
        - scoring: method of scoring
        """
        self.max_features = max_features
        if min_features is None:
            self.min_features = 1
        else:
            self.min_features = min_features
    
        self.model = model
        self.scoring = scoring
        self.cv = cv
        self.ftr_ = []
        return
    
    def fit(self, X, y):
        """
        fits the ForwardSelection model
        
        parameters:
        - X: features and their values
        - y: targets and their values
        
        returns: a list of indexes of the features to be selected        
        """
        best_error = np.Inf
        
        if X.shape[1] >= 1 and X.shape[0] >= X.shape[1]:
            while True:
            
                # get errors from new_candidates and create a list of errors with mean of cross_val_score 
                errors = []
                new_candidates = list(set(range(X.shape[1])) - set(self.ftr_))
                
                # for each index in new_candidates, add each index with the existing indexes and get error for each
                for i in new_candidates:
                    X_new = X[:,self.ftr_ + [i]]
                    error = 1 - np.mean(cross_val_score(self.model, X_new, y, cv= self.cv))
                    errors.append(error)
            
                # check 2 termination coniditions: (1) if error is too large, (2) if we exceed max features
                # if we can't terminate or we have under the min features, set new best_error and add index of min_error to self.ftr_
                if np.min(errors) >=  best_error:
                    if len(self.ftr_) < self.min_features:
                        best_error = np.min(errors)
                        self.ftr_.append(new_candidates[np.argmin(errors)])
                        print("need more features")
                    else:
                        break
                
                elif len(self.ftr_) >= self.max_features:
                        break    
                else:
                    best_error = np.min(errors)
                    self.ftr_.append(new_candidates[np.argmin(errors)])
                    print("can't terminate, adding another feature")
                    
                    
    def transform(self, X, y=None):
        """
        transforms a given X split to select a subset of features
        
        parameters:
        - X: features and their values
        - y: is equal to None because we are selecting features, not targets
        """
        return X[:, self.ftr_]

In [ ]:
# forward selection implementation

from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV, cross_val_score

fs_1 = ForwardSelection(LinearRegression(), min_features = 1, max_features = 10, cv = 10)
fs_1.fit(X_train, y_train)
X_train_sel = fs_1.transform(X_train)
X_test_sel = fs_1.transform(X_test)

clf = SVR(kernel = 'rbf')
errors = fit_and_report(clf, X_train_sel, y_train, X_test_sel, y_test, mode = "classification")

print('Train error %0.3f'%(errors[0]))
print('Test error %0.3f'%(errors[1]))